In [10]:
from evaluate import manual_eval, compute_per_class_iou
import json
import numpy as np
from tqdm import tqdm

In [11]:
conditions = ["normal", "dark", "rain", "snow", "fog", "fog_rain", "dark_denoised", "rain_denoised", "snow_denoised", "fog_denoised", "fog_rain_denoised"]
dt_list = [
    "FastRCNNWrapper",
    "FasterRCNN",
    "MaskRCNN",
    "SSD",
    "WrappedDETR",
    "WrappedYOLO",
]

result_paths = {}
for condition in conditions:


    gt_path = "data/sets/nuimages/nuimages_200.json"


    result_paths[condition] = {}
    result_paths[condition]["gt_path"] = gt_path

    if condition == "normal":
        result_paths[condition]["paths"] = [f"outputs/autoencoder/{i}_results_clean.json" for i in dt_list]

    elif condition.endswith("_denoised"):
        result_paths[condition]["paths"] = [f"outputs/autoencoder/{i}_results_{condition.split("_")[0]}_3.json" for i in dt_list]
    else:
        result_paths[condition]["paths"] = [f"outputs/autoencoder_no_denoised/{i}_results_{condition}_3.json" for i in dt_list]



In [12]:
result_paths

{'normal': {'gt_path': 'data/sets/nuimages/nuimages_200.json',
  'paths': ['outputs/autoencoder/FastRCNNWrapper_results_clean.json',
   'outputs/autoencoder/FasterRCNN_results_clean.json',
   'outputs/autoencoder/MaskRCNN_results_clean.json',
   'outputs/autoencoder/SSD_results_clean.json',
   'outputs/autoencoder/WrappedDETR_results_clean.json',
   'outputs/autoencoder/WrappedYOLO_results_clean.json']},
 'dark': {'gt_path': 'data/sets/nuimages/nuimages_200.json',
  'paths': ['outputs/autoencoder_no_denoised/FastRCNNWrapper_results_dark_3.json',
   'outputs/autoencoder_no_denoised/FasterRCNN_results_dark_3.json',
   'outputs/autoencoder_no_denoised/MaskRCNN_results_dark_3.json',
   'outputs/autoencoder_no_denoised/SSD_results_dark_3.json',
   'outputs/autoencoder_no_denoised/WrappedDETR_results_dark_3.json',
   'outputs/autoencoder_no_denoised/WrappedYOLO_results_dark_3.json']},
 'rain': {'gt_path': 'data/sets/nuimages/nuimages_200.json',
  'paths': ['outputs/autoencoder_no_denoised/Fa

In [13]:
eval_result = {}
for i in dt_list:
    dt_normal: str = [s for s in result_paths["normal"]["paths"] if i in s][0]
    gt_normal: str = result_paths["normal"]["gt_path"]
    try:
        normal_TP, normal_FP, normal_FN, normal_precision, normal_recall, normal_IoU = manual_eval(gt_json_path=gt_normal, dt_json_path=dt_normal)
    except FileNotFoundError:
        tqdm.write(f"{i} has no clean result ({dt_normal})")
        continue


    normal_F1_score = 2 * ((normal_precision * normal_recall) / (normal_precision + normal_recall))

    eval_result[i] = {}
    eval_result[i]["normal_precision"] = normal_precision
    eval_result[i]["normal_recall"] = normal_recall
    eval_result[i]["normal_TP"] = normal_TP
    eval_result[i]["normal_FP"] = normal_FP
    eval_result[i]["normal_FN"] = normal_FN
    eval_result[i]["F1_score"] = normal_F1_score
    eval_result[i]["mean_IoU"] = normal_IoU

    for condition in [k for k in result_paths if k != "normal"]:
        RS = 0
        RS_F1 = 0
        F1_score_list = []
        rs_list = []
        recall_list = []
        precision_list = []
        iou_list = []

        dt_corrupt = [s for s in result_paths[condition]["paths"] if i in s][0]
        gt_corrupt = result_paths[condition]["gt_path"]

        try:
            corrupt_TP, corrupt_FP, corrupt_FN, corrupt_precision, corrupt_recall, corrupt_IoU = manual_eval(gt_json_path=gt_corrupt, dt_json_path=dt_corrupt)
        except FileNotFoundError:
            corrupt_precision = 0.0
            corrupt_recall = 0.0
            corrupt_IoU = 0.0
            corrupt_TP = corrupt_FP = corrupt_FN = 0

        corrupt_F1_score = ((corrupt_precision * corrupt_recall) / (corrupt_precision + corrupt_recall)) if corrupt_precision + corrupt_recall != 0 else 0

        RS = RS + corrupt_precision
        RS_F1 = RS_F1 + corrupt_F1_score
        rs_list.append(RS)
        F1_score_list.append(corrupt_F1_score)
        precision_list.append(corrupt_precision)
        recall_list.append(corrupt_recall)
        iou_list.append(corrupt_IoU)

        RS = RS / (1 * normal_precision)
        RS_F1 = RS_F1 / (1 * normal_F1_score)

        # tqdm.write(f"Resilience Score (RS) of {i} for {corruption} corruption = {RS_F1} | AP (iou:0.5) = {RS_F1 * clean_precision}")
        

        eval_result[i][condition] = {
            "TP": corrupt_TP,
            "FP": corrupt_FP,
            "FN": corrupt_FN,
            "precision": RS * normal_precision,
            "recall": np.mean(recall_list),
            "F1_score": np.mean(F1_score_list),
            "IoU": np.mean(iou_list),
            "RS": RS_F1,
        }
    

In [14]:
eval_result

{'FastRCNNWrapper': {'normal_precision': 0.7953964194373402,
  'normal_recall': 0.24488188976377953,
  'normal_TP': 311,
  'normal_FP': 80,
  'normal_FN': 959,
  'F1_score': 0.37447320891029506,
  'mean_IoU': np.float64(0.8124707955619402),
  'dark': {'TP': 283,
   'FP': 73,
   'FN': 987,
   'precision': 0.7949438202247191,
   'recall': np.float64(0.22283464566929134),
   'F1_score': np.float64(0.17404674046740468),
   'IoU': np.float64(0.8137835670283402),
   'RS': 0.46477754970475743},
  'rain': {'TP': 234,
   'FP': 64,
   'FN': 1036,
   'precision': 0.785234899328859,
   'recall': np.float64(0.18425196850393702),
   'F1_score': np.float64(0.14923469387755103),
   'IoU': np.float64(0.8247097712274322),
   'RS': 0.39851901371481063},
  'snow': {'TP': 224,
   'FP': 50,
   'FN': 1046,
   'precision': 0.8175182481751825,
   'recall': np.float64(0.1763779527559055),
   'F1_score': np.float64(0.1450777202072539),
   'IoU': np.float64(0.8167573776778269),
   'RS': 0.38741815637339017},
  'f

In [15]:
with open("eval_result_200_denoised.json", 'w') as json_file:
    json.dump(eval_result, json_file, indent=4)

In [16]:
eval_per_class_result = {}
for i in tqdm(dt_list):
    dt_normal: str = [s for s in result_paths["normal"]["paths"] if i in s][0]
    gt_normal: str = result_paths["normal"]["gt_path"]

    try:
        result_clean = compute_per_class_iou(gt_json_path=gt_normal, dt_json_path=dt_normal)
    except FileNotFoundError:
        print(f"{i} has no clean result ({dt_normal})")
        continue

    eval_per_class_result[i] = {}
    eval_per_class_result[i]["clean"] = result_clean

    for condition in [k for k in result_paths if k != "normal"]:

        dt_corrupt = [s for s in result_paths[condition]["paths"] if i in s][0]
        gt_corrupt = result_paths[condition]["gt_path"]

        try:
            corrupt_result = compute_per_class_iou(gt_json_path=gt_corrupt, dt_json_path=dt_corrupt)
        except FileNotFoundError:
            corrupt_result = None

        eval_per_class_result[i][condition] = corrupt_result

100%|██████████| 6/6 [00:00<00:00, 14.54it/s]


In [17]:
eval_per_class_result

{'FastRCNNWrapper': {'clean': [{'category_id': 2,
    'category_name': 'bicycle',
    'TP': 14,
    'FP': 5,
    'FN': 29,
    'Precision': 0.7368,
    'Recall': 0.3256,
    'Mean IoU@0.50 (TP only)': 0.7982,
    'TP matched count': 14},
   {'category_id': 3,
    'category_name': 'bus',
    'TP': 4,
    'FP': 5,
    'FN': 19,
    'Precision': 0.4444,
    'Recall': 0.1739,
    'Mean IoU@0.50 (TP only)': 0.9111,
    'TP matched count': 4},
   {'category_id': 4,
    'category_name': 'car',
    'TP': 183,
    'FP': 34,
    'FN': 439,
    'Precision': 0.8433,
    'Recall': 0.2942,
    'Mean IoU@0.50 (TP only)': 0.8399,
    'TP matched count': 183},
   {'category_id': 6,
    'category_name': 'motorcycle',
    'TP': 11,
    'FP': 3,
    'FN': 35,
    'Precision': 0.7857,
    'Recall': 0.2391,
    'Mean IoU@0.50 (TP only)': 0.7485,
    'TP matched count': 11},
   {'category_id': 1,
    'category_name': 'person',
    'TP': 73,
    'FP': 25,
    'FN': 352,
    'Precision': 0.7449,
    'Recall': 

In [18]:
with open("eval_per_class_result_200_denoised.json", 'w') as json_file:
    json.dump(eval_per_class_result, json_file, indent=4)